In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
samples_per_class = 5000
no_of_classes = 3
directory = "/content/gdrive/MyDrive/GW_SOP/Data/"

In [ ]:
noise_df = pd.read_csv(directory + "Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)


data_BBH_df = pd.read_csv(directory + "Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)


data_BNS_df = pd.read_csv(directory + "Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BNS = data_BNS_df.values.astype(float)

In [ ]:
noise *= 1e19
data_BBH *= 1e19
data_BNS *= 1e20

In [ ]:
X = np.concatenate((noise, data_BBH, data_BNS), axis=0)
print(X.shape)

In [ ]:
y = [int(i/samples_per_class) for i in range(samples_per_class*no_of_classes)]
y = tf.keras.utils.to_categorical(y)
print(y)

In [ ]:
X = np.expand_dims(X, axis=-1)
print(X.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1)

In [ ]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
model = Sequential()

model.add(Conv1D(64, 32, input_shape = (16384,1)))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(128, 64))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(256, 64))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(512, 128))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))                         
model.add(Dense(3, activation=tf.nn.softmax))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10) #validation_data=(X_val, y_val)

In [ ]:
acc = [0.] + history.history['accuracy']

loss = history.history['loss']

plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training Accuracy')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.legend(loc='upper right')
plt.ylabel('Categorical Cross Entropy')
plt.ylim([0,2])
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()